In [1]:
import import_ipynb, re, requests, json
import Authentication
from config import ServerConfig
from config import AuthConfig

hash_list = []
server_list = []
notList = []
url_pattern = re.compile("^(http:\/\/|https:\/\/)+[a-z0-9]+([\-\.]{1}[a-z0-9]+)*\.[a-z]{2,5}(:[0-9]{1,5})?(\/.*)?(?<!\/)$")
for server in ServerConfig.server_list:
    if url_pattern.match(server[1]):
        server_list.append(Authentication.Server(server[0], server[1]))
    else:
        if server[1].endswith("/"):
            raise Exception('Improper URL found (URL ends with slash). URL: {}'.format(server[1]))
        else:
            raise Exception('Improper URL found. URL: {}'.format(server[1]))

events_api = "/fapi/query/hunt-v2/events?"


server = server_list[server_list.index("S1 Server")]
Authentication.load_authentication(server)

importing Jupyter notebook from Authentication.ipynb


True

In [2]:
print(server_list[0])

S1 Server


In [3]:
print(server_list[0].url)

https://s1.managedpdr.csintelligence.asia


In [4]:
server_list[0].url + events_api

'https://s1.managedpdr.csintelligence.asia/fapi/query/hunt-v2/events?'

In [5]:

def getResponse(eventTypeArray, startTime, endTime,orQueryList, notQueryList):
    networkRelatedEvents ={"query":{"andQuery":[["$$maia$$.eventType",eventTypeArray],["$$maia$$.happenedBefore", startTime], ["$$maia$$.happenedAfter", endTime]],"orQuery":orQueryList,"notQuery":notQueryList,"huntQueryVersion":"1.0.0"}}
    
    response = requests.post(server_list[0].url + events_api, json = networkRelatedEvents
, headers=server.header, cookies=server.cookie)
  #  print(json.loads(response.text)["remainingItems"])
    return json.loads(response.text)


In [6]:
def getSignersInPage():
    signer_list = []
    signerFlag = False
    signerArrayIndex = 0
    jsonResult = getResponse([8], "2020-02-13T15:59:59.316Z", "2020-02-12T16:00:00.316Z",[["$$maia$$.filename",["*"]]], notList)
    for event in jsonResult["result"]:
        if "certInfo" in event["payload"]["process"]["program"]:
            curr_signer = event["payload"]["process"]["program"]["certInfo"]["signer"]
            if curr_signer not in signer_list:
                signer_list.append(curr_signer)
                
    return signer_list
       # elif "remoteAddr" in event["payload"]["data"]:
       #     if event["payload"]["data"]["outbound"] == False:
       #         ipList.append(event["payload"]["data"]["localAddr"])
       #     else:
       #         ipList.append(event["payload"]["data"]["remoteAddr"])
       #     
       # elif "queryName" in event["payload"]["data"]:
       #     dnsList.append(event["payload"]["data"]["queryName"])   
        
    #if len(signer_list) > 0 and signerFlag == False:
    #    notList.append(["signer",[]])
    #    signerArrayIndex = len(notList) - 1
    #    signerFlag = True
    
    #if len(ipList) > 0 and ipFlag == False:
    #    notList.append(["ip.dst",[]])
    #    ipArrayIndex = len(notList) - 1
    #    ipFlag = True
    #    
    #if len(dnsList) > 0 and dnsFlag == False:
    #    notList.append(["dns",[]])
    #    dnsArrayIndex = len(notList) - 1
    #    dnsFlag = True
    #    
    #for signer in signer_list:
    #    if signer not in notList[signerArrayIndex][1]:
    #        notList[signerArrayIndex][1].append(signer)
    #for ip in ipList:
    #    if ip not in notList[ipArrayIndex][1]:
    #        notList[ipArrayIndex][1].append(ip)
    #for dns in dnsList:
    #    if dns not in notList[dnsArrayIndex][1]:
    #        notList[dnsArrayIndex][1].append(dns)
            
    #return jsonResult["remainingItems"]

In [7]:
print(getSignersInPage())

['Microsoft Windows']


In [13]:
def getAllEventsSinglePageSigner(signer, notQuery = []):
    notQueryList = []
    md5List = []
    sha1List = []
    sha256List = []
    md5Index = None
    sha1Index = None
    sha256Index = None
    
    if notQuery != []:    
        for field in notQuery:
            if field[0] == "$$maia$$.md5":
                md5Index = notQuery.index(field)
            elif field[0] == "$$maia$$.sha1":
                sha1Index = notQuery.index(field)
            elif field[0] == "$$maia$$.sha256":
                sha256Index = notQuery.index(field)
    
    if md5Index != None:
        md5List = notQuery[md5Index][1]
    if sha1Index != None:
        sha1List = notQuery[sha1Index][1]
    if sha256Index != None:
        sha256List = notQuery[sha256Index][1]
        
    jsonResult = getResponse([8], "2020-02-13T15:59:59.316Z", "2020-02-12T16:00:00.316Z",[["signer",[signer]]], notQuery)
    for event in jsonResult["result"]:
        print(event)
        break
        if "process" in event["payload"]:
            if "md5" in event["payload"]["process"]["program"]:
                if event["payload"]["process"]["program"]["md5"] not in md5List:
                    md5List.append(event["payload"]["process"]["program"]["md5"])
            elif "sha1" in event["payload"]["process"]["program"]:
                if event["payload"]["process"]["program"]["sha1"] not in sha1List:
                    sha1List.append(event["payload"]["process"]["program"]["sha1"])
            elif "sha256" in event["payload"]["process"]["program"]:
                if event["payload"]["process"]["program"]["sha256"] not in sha256List:
                    sha256List.append(event["payload"]["process"]["program"]["sha256"])
    
    if md5List != []:
        notQueryList.append(["$$maia$$.md5", md5List])
    if sha1List != []:
        notQueryList.append(["$$maia$$.sha1", sha1List])
    if sha256List != []:
        notQueryList.append(["$$maia$$.sha256", sha256List])
    
    #print(md5List)
    return notQueryList, jsonResult["remainingItems"]
    #getAllHashesFromSigner(signer, notQueryList)
    

In [9]:
def getAllEventsUnderSigner(signer):
    remaining = 1
    notList = []
    while remaining > 0:
        notList, remaining = getAllEventsSinglePageSigner(signer, notList)
    return notList

In [10]:
def getIndividualHashCount(hashType, hashValue):
    jsonResult = getResponse([8], "2020-02-13T15:59:59.316Z", "2020-02-12T16:00:00.316Z",[[hashType,[hashValue]]], [])
    if jsonResult["remainingItems"] > 0:
        return jsonResult["remainingItems"] + 20
    else:
        return len(jsonResult["result"])

In [11]:
def getSignerHashListCounts(hashList):
    for hashset in hashList:
        for eachHash in hashset[1]:
            print(eachHash + ":" + str(getIndividualHashCount(hashset[0], eachHash)))

In [12]:
getSignerHashListCounts(getAllFileNameHashesUnderSigner("Google Inc"))

750446ed76a5d13e902174dddda1a62b:18
dd7423abbe2913e70d50e9318ad57ee4:9
79b804e8a81bfd9c6a3749b4f3ee86e2:13
a8fd9222e4d72596bb37da8be95c0ba4:6
d815683853a11eb0dbeec3941c11c75d:2
605ccc9ce1839bc5583017df7cae27a6:7
0545a3eb959cfa4790d267bfb8c1aca4:3


In [14]:
getAllEventsSinglePageSigner("Google Inc")

{'eventId': '566270928325967873', 'endpointId': '512502869702737920', 'payload': {'localId': '566270928170780673', 'process': {'id': '512502869702737920:7400:1581609509691', 'parentId': '512502869702737920:904:1580785685628', 'endpointId': '512502869702737920', 'program': {'path': 'c:\\users\\dickson.teo\\appdata\\local\\google\\update\\googleupdate.exe', 'filename': 'googleupdate.exe', 'md5': '750446ed76a5d13e902174dddda1a62b', 'sha1': '9d04597f8cfc8841dfa876487de965c0f05708ca', 'sha256': 'f67355a6659e21d8d97e6982b28f22453f8c298e822e27faddb440da4a6de7c0', 'certInfo': {'signer': 'Google Inc', 'issuer': 'VeriSign Class 3 Code Signing 2010 CA', 'trusted': True, 'expired': True}, 'size': 154440, 'arch': 'x32', 'fsName': 'googleupdate.exe'}, 'user': 'HGPSVR\\dickson.teo', 'pid': 7400, 'startTime': '2020-02-13T15:58:29.691Z', 'ppid': 904, 'pstartTime': '2020-02-04T03:08:05.628Z', 'userSID': 'S-1-5-21-1545398807-1586680928-1461412557-1660', 'privilegeLevel': 'MEDIUM', 'noGui': True}, 'incide

([], 38)